In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the dataset
df = pd.read_csv('/content/Indian_Food_Dataset_With_Recipes.csv')
df.head()

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,Cleaned-Ingredients,image-url,Ingredient-count,FullRecipe
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"salt,amchur (dry mango powder),karela (bitter ...",https://www.archanaskitchen.com/images/archana...,10,🧂 Ingredients:\n1 tablespoon Red Chilli powder...
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon ...",15,South Indian Recipes,"To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,"tomato,salt,chickpea lentils,green chilli,rice...",https://www.archanaskitchen.com/images/archana...,12,🧂 Ingredients:\n 2 teaspoon cashew - or peanut...
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1 Onion - sliced,1 teaspoon White Urad Dal (Sp...",50,South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi...",https://www.archanaskitchen.com/ragi-vermicell...,"salt,rice vermicelli noodles (thin),asafoetida...",https://www.archanaskitchen.com/images/archana...,12,"🧂 Ingredients:\n1 Onion - sliced,1 teaspoon Wh..."
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"1/2 teaspoon Turmeric powder (Haldi),1 tablesp...",45,Andhra,To begin making Gongura Chicken Curry Recipe f...,https://www.archanaskitchen.com/gongura-chicke...,"tomato,salt,ginger,sorrel leaves (gongura),fen...",https://www.archanaskitchen.com/images/archana...,15,🧂 Ingredients:\n1/2 teaspoon Turmeric powder (...
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"oil - as per use, 1 tablespoon coriander seed...",30,Andhra,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,"tomato,salt,ginger,red chillies,curry,asafoeti...",https://www.archanaskitchen.com/images/archana...,12,"🧂 Ingredients:\n oil - as per use, 1 tablespoo..."


In [ ]:
# Drop missing values in key fields
df = df.dropna(subset=['TranslatedIngredients', 'TranslatedRecipeName', 'TranslatedInstructions'])

# Initialize sentence transformer for retrieval
retriever = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for each recipe's ingredients
ingredient_embeddings = retriever.encode(df['TranslatedIngredients'].tolist(), convert_to_tensor=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
def retrieve_top_k_recipes(user_ingredients, k=3):
    user_embedding = retriever.encode([user_ingredients], convert_to_tensor=True)
    scores = cosine_similarity(user_embedding, ingredient_embeddings)[0]
    top_indices = scores.argsort()[-k:][::-1]

    return df.iloc[top_indices]

In [ ]:
# Load GPT-2 model & tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
def generate_response(recipes_df, user_ingredients):
    prompt = f"You have the following ingredients: {user_ingredients}\n\nHere are some Indian recipes you can make:\n\n"

    for idx, row in recipes_df.iterrows():
        recipe_info = (
            f"🍲 Recipe: {row['TranslatedRecipeName']}\n"
            f"⏱ Total Time: {row.get('TotalTimeInMins', 'N/A')} mins\n"
            f"📖 Instructions: {row['TranslatedInstructions'][:300]}...\n"
            f"🌿 Ingredients: {row['TranslatedIngredients'][:100]}...\n"
            f"🍽 Cuisine: {row['Cuisine']}\n\n"
        )
        prompt += recipe_info

    inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(inputs, max_length=1024, temperature=0.7, pad_token_id=tokenizer.eos_token_id)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
def rag_recipe_suggester(user_ingredients):
    top_recipes = retrieve_top_k_recipes(user_ingredients)
    generated = generate_response(top_recipes, user_ingredients)
    return generated

In [ ]:
user_input = "onion, tomato, garlic, green chili, turmeric, cumin seeds"
response = rag_recipe_suggester(user_input)
print(response)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


You have the following ingredients: onion, tomato, garlic, green chili, turmeric, cumin seeds

Here are some Indian recipes you can make:

🍲 Recipe: Pudina Rasam Recipe - Mint Leaves Rasam
⏱ Total Time: 20 mins
📖 Instructions: To begin making Pudina Rasam Recipe, soak tamarind in warm water for 15 minutes and extract the juice.Crush cumin seeds and pepper seeds in a mortar and pestle.
Add this to the tamarind juice and keep aside.
 To this tamarind mixture add the chopped tomato and mash it well into the water.
In a mixer...
🌿 Ingredients: Salt - as required,1/2 teaspoon Whole Black Peppercorns,3 Dry Red Chillies,2 teaspoon Sunflower Oil,...
🍽 Cuisine: Tamil Nadu

🍲 Recipe: Pita Saga Bati Basa Recipe
⏱ Total Time: 25 mins
📖 Instructions: To begin making the Pita Saga Bati Basa recipe, take a deep frying pan or wok.
Roast the badis (sun-dried lentil dumpling) and crush roughly into small pieces.Now chop onion finely and keep aside.
Next cut eggplant, potato and tomato into smaller cubes

In [ ]:
def generate_response(recipes_df, user_ingredients, max_recipes=2):
    prompt = f"You have the following ingredients: {user_ingredients}\n\nHere are some Indian recipes you can make:\n\n"

    for idx, row in recipes_df.head(max_recipes).iterrows():
        recipe_info = (
            f"🍲 Recipe: {row['TranslatedRecipeName']}\n"
            f"⏱ Total Time: {row.get('TotalTimeInMins', 'N/A')} mins\n"
            f"📖 Instructions: {row['TranslatedInstructions'][:300]}...\n"
            f"🌿 Ingredients: {row['TranslatedIngredients'][:100]}...\n"
            f"🍽 Cuisine: {row['Cuisine']}\n\n"
        )
        prompt += recipe_info

    inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(inputs, max_length=1024, temperature=0.3, pad_token_id=tokenizer.eos_token_id)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
while True:
    user_input = input("\n📝 Enter the ingredients you have (or type 'exit' to quit):\n> ")
    if user_input.lower() == 'exit':
        print("👋 Exiting. Happy cooking!")
        break

    top_recipes = retrieve_top_k_recipes(user_input, k=2)  # retrieve more, filter fewer
    response = generate_response(top_recipes, user_input, max_recipes=2)
    print("\n🍽️ Suggested Recipes:\n")
    print(response)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🍽️ Suggested Recipes:

You have the following ingredients: chicken, salt, eggs

Here are some Indian recipes you can make:

🍲 Recipe: Hash Browns With Chives Recipe
⏱ Total Time: 40 mins
📖 Instructions: To begin making Green Tomato Hash Browns And Egg Poach Recipe, first peel the potatoes and grate it into a bowl.
Rest the grated potatoes for ten minutes and squeeze out the juice from the potatoes.
 Cut the bread in tiny pieces and add to potatoes.
You can also tear up the bread and add them.
Add i...
🌿 Ingredients: Sea salt - to taste,5 Cheddar cheese - cubes,4 Potatoes (Aloo),1 Whole Egg - beaten,2 Tomato - thick...
🍽 Cuisine: Continental

🍲 Recipe: Chicken Crepe Lasagne Recipe
⏱ Total Time: 35 mins
📖 Instructions: To begin making the Chicken Crepe Lasagne recipe,we need to make the crepe batter.
To make the crepe batter, in a mixing bowl, combine whole wheat flour, sooji, salt to taste, curdAdd water as you mix.
The batter should be thinner than pancake batter.In a flat griddle, ma